In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import re
import numpy as np
import pandas as pd
import math
import pickle
from collections import Counter
import unicodedata

In [ ]:
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
Stopwords = set(stopwords.words('english'))
ps=PorterStemmer()

In [ ]:
data = pd.read_csv('/content/English.csv')

In [ ]:
def finding_all_unique_words_and_freq(words):
    words_unique = []
    word_freq = {}
    for word in words:
        if word not in words_unique:
            words_unique.append(word)
    for word in words_unique:
        word_freq[word] = words.count(word)
    return word_freq
def finding_freq_of_word_in_doc(word,words):
    freq = words.count(word)

def remove_special_characters(text):
    regex = re.compile('[^a-zA-Z\s]')
    text_returned = re.sub(regex,' ',text)
    return text_returned
def strip_accents(text):
    return ''.join(c for c in unicodedata.normalize('NFKD', text) if unicodedata.category(c) != 'Mn')

In [ ]:
doc_words=[]
dict_global = {}
idx = 0
files_with_index = {}
for index in range(len(data)):
    fname=str(data.loc[index]['Chapter'])+"."+str(data.loc[index]['Verse'])
    sloka=data.loc[index]['Sloke']
    text=data.loc[index]['Commentary']
    text=strip_accents(text)
    text = remove_special_characters(text)
    words = word_tokenize(text)
    words = [word for word in words if len(words)>1]
    words = [word.lower() for word in words]
    words = [ps.stem(word) for word in words]
    words = [word for word in words if word not in Stopwords]
    doc_words.append(words)
    dict_global.update(finding_all_unique_words_and_freq(words))
    files_with_index[idx] = fname
    idx = idx + 1

unique_words_all = set(dict_global.keys())

In [ ]:
tf={}
df={}
for i in unique_words_all:
    tf[i]={}
    df[i]=0

In [ ]:
idx=0
Ltot=0
Ld={}
for index in range(len(data)):
    text=data.loc[index]['Commentary']
    text=strip_accents(text)
    text = remove_special_characters(text)
    words = word_tokenize(text)
    words = [word.lower() for word in words]
    words=[ps.stem(word) for word in words]
    Ld[idx]=len(words) #len of current doc
    Ltot=Ltot+len(words) #sum of lens of all the docs
    words=[word for word in words if word not in Stopwords]
    counter=Counter(words)
    for i in counter.keys():
        df[i]=df[i]+1
        tf[i][idx]=counter[i]
    idx=idx+1

In [ ]:
doc_norm={}
idx=0
for i in doc_words:
    l2=0
    for j in set(i):
        l2+=(i.count(j)*math.log(len(files_with_index)/df[j]))**2
    doc_norm[idx]=(math.sqrt(l2))
    idx+=1

In [ ]:
with open('pickles/posting_list.pkl','wb') as file:
    pickle.dump(tf,file)
    file.close()

with open('pickles/df.pkl','wb') as file:
    pickle.dump(df,file)
    file.close()

with open('pickles/doc_len.pkl','wb') as file:
    pickle.dump(Ld,file)
    file.close()

In [ ]:
a_file = open("pickles/file_idx.pkl", "wb")
pickle.dump(files_with_index, a_file)
a_file.close()
a_file = open("pickles/unique_words_all.pkl", "wb")
pickle.dump(unique_words_all , a_file)
a_file.close()

In [ ]:
with open('pickles/doc_words.pkl','wb') as file:
    pickle.dump(doc_words,file)
    file.close()

with open('pickles/doc_norm.pkl','wb') as file:
    pickle.dump(doc_norm,file)
    file.close()